In [ ]:
import torch
from torch.utils.data import DataLoader
import importlib

In [2]:
# helper function for colab -- ugh
def load_py(filename, alias: str = None):
    """Handle importing and reloading modules when in colab"""
    import requests, importlib, sys
    if not filename.endswith(".py"): filename += ".py"
    
    if 'google.colab' in sys.modules:
        if '/content/' not in sys.path:
            sys.path.insert(0, '/content/')

        url = f"https://raw.githubusercontent.com/BenAF002/dabble_bot_v2_dev/refs/heads/main/phonemes/{filename}"
        code = requests.get(url).text

        # write to /content for colab
        destination = f"/content/{filename}"
        with open(destination, "w") as f:
            f.write(code)

    # handle module reloading
    module_name = filename[:-3] # Strip the .py extension
    if alias in globals():
        importlib.reload(globals()[alias])
    elif module_name in globals():
        importlib.reload(globals()[module_name])
    else:
        if alias is not None:
            globals()[alias] = importlib.import_module(module_name)
        else:
            globals()[module_name] = importlib.import_module(module_name)

In [3]:
modules = ["encoder", "decoder", "dataset", "train"]
for module in modules:
    load_py(module + ".py")

In [4]:
wd = dataset.Words()
train_data, test_data = wd.train_test_split(0.8)

train_data.__len__(), test_data.__len__()

(93131, 23283)

## BERT Encoder

In [5]:
bert = encoder.EncoderBERT(n_emb=64, exp=4, n_heads=2, n_blocks=2, dropout=0.0, pool='cls', mask_attn=True)

print(f"n params: {sum(p.numel() for p in bert.parameters()):,.0f}")
if bert.device == 'cuda': 
    bert.to('cuda')
    bert = torch.compile(bert)

n params: 114,610


In [6]:
# try training
optimizer = torch.optim.AdamW(bert.parameters(), lr=6e-4)

train.training_loop(
    model=bert, 
    train_data=train_data, 
    test_data=test_data,
    optimizer=optimizer,
    batch_size=256,
    n_epochs=40, 
)

/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1551: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  return _C._get_float32_matmul_precision()
W1217 01:44:15.258000 3324 torch/_inductor/utils.py:1558] [0/0] Not enough SMs to use max_autotune_gemm mode


Epoch 1/40 - Train Loss: 0.3616, Test Loss: 0.2194 | ETA: 30 min, 14 sec
Epoch 2/40 - Train Loss: 0.2027, Test Loss: 0.1934 | ETA: 15 min, 30 sec
Epoch 3/40 - Train Loss: 0.1855, Test Loss: 0.1805 | ETA: 10 min, 39 sec
Epoch 4/40 - Train Loss: 0.1733, Test Loss: 0.1680 | ETA: 8 min, 9 sec
Epoch 5/40 - Train Loss: 0.1610, Test Loss: 0.1566 | ETA: 6 min, 38 sec
Epoch 6/40 - Train Loss: 0.1504, Test Loss: 0.1475 | ETA: 5 min, 37 sec
Epoch 7/40 - Train Loss: 0.1420, Test Loss: 0.1396 | ETA: 4 min, 52 sec
Epoch 8/40 - Train Loss: 0.1349, Test Loss: 0.1339 | ETA: 4 min, 18 sec
Epoch 9/40 - Train Loss: 0.1287, Test Loss: 0.1276 | ETA: 3 min, 51 sec
Epoch 10/40 - Train Loss: 0.1231, Test Loss: 0.1221 | ETA: 3 min, 29 sec
Epoch 11/40 - Train Loss: 0.1178, Test Loss: 0.1170 | ETA: 3 min, 10 sec
Epoch 12/40 - Train Loss: 0.1137, Test Loss: 0.1135 | ETA: 2 min, 54 sec
Epoch 13/40 - Train Loss: 0.1101, Test Loss: 0.1098 | ETA: 2 min, 40 sec
Epoch 14/40 - Train Loss: 0.1071, Test Loss: 0.1083 | ETA:

## GRU Decoder

In [4]:
importlib.reload(phon.decoder)

gru = phon.decoder.DecoderGRU(input_size=(64+50), hidden_size=256)
gru.to(torch.device('cuda'))
gru = torch.compile(gru)
params_devices = {p.device for p in gru.parameters()}
print("model param devices:", params_devices)
print(f"n params: {sum(p.numel() for p in gru.parameters()):,.0f}")

model param devices: {device(type='cuda', index=0)}
n params: 295,968


In [5]:
optimizer = torch.optim.AdamW(gru.parameters(), lr=6e-4)

importlib.reload(phon.train)
phon.train.training_loop(
    model=gru, 
    train_data=train, 
    test_data=test, 
    optimizer=optimizer,
    batch_size=256,
    n_epochs=1, 
)

torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])


W1216 19:30:05.520000 22364 site-packages\torch\_inductor\utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.

In [7]:
dl = DataLoader(test, batch_size=32, shuffle=True)
yb, xb = next(iter(dl))
yb, xb = yb.to(gru.device), xb.to(gru.device)

for i in range(10):
    print(
        wd.dec(yb[i]), ' : ',
        wd.dec(gru.predict(xb[i].unsqueeze(1).T)[0])
    )

guzek  :  gueg
detach  :  datidt
cavagnaro  :  arkowango
creech  :  kurie
foose  :  faues
boasting  :  boottyming
discussed  :  decesfud
fredericksen  :  vertircon
knickknacks  :  cnouricac
washington  :  awithon


## Transformer Decoder

TODO